In [ ]:
import erlab.plotting.erplot as eplt
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from erlab.io.exampledata import generate_data

In [ ]:
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96
plt.rcParams["image.cmap"] = "viridis"
plt.rcParams["figure.figsize"] = eplt.figwh(wscale=1.2, fixed_height=False)

nb_execution_mode = "cache"

Let's start by defining a model function and the data to fit.

In [ ]:
def poly1(x, a, b):
    return a * x + b


# Generate some toy data
x = np.linspace(0, 10, 20)
y = poly1(x, 1, 2)

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(1)
yerr = np.full_like(x, 0.5)
y = rng.normal(y, yerr)

In [ ]:
import lmfit

model = lmfit.Model(poly1)
params = model.make_params(a=1.0, b=2.0)
result = model.fit(y, x=x, params=params, weights=1 / yerr)

result.plot()
result

By passing dictionaries to `make_params`, we can set the initial values of the parameters and also set the bounds for the parameters.

In [ ]:
model = lmfit.Model(poly1)
params = model.make_params(
    a={"value": 1.0, "min": 0.0},
    b={"value": 2.0, "vary": False},
)
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

In [ ]:
result.params

In [ ]:
result.params["a"].value, result.params["a"].stderr

The parameters can also be retrieved in a form that allows easy error propagation calculation, enabled by the [uncertainties](https://github.com/lmfit/uncertainties) package.

In [ ]:
a_uvar = result.uvars["a"]
print(a_uvar)
print(a_uvar**2)

In [ ]:
# Generate toy data
x = np.linspace(0, 10, 50)
y = -0.1 * x + 2 + 3 * np.exp(-((x - 5) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(x, 0.3)
y = rng.normal(y, yerr)

# Plot the data
plt.errorbar(x, y, yerr, fmt="o")

A composite model can be created by adding multiple models together.

In [ ]:
from lmfit.models import GaussianModel, LinearModel

model = GaussianModel() + LinearModel()
params = model.make_params(slope=-0.1, center=5.0, sigma={"value": 0.1, "min": 0})
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
result.plot()
result

How about multiple gaussian peaks? Since the parameter names overlap between the models, we must use the `prefix` argument to distinguish between them.

In [ ]:
model = GaussianModel(prefix="p0_") + GaussianModel(prefix="p1_") + LinearModel()
model.make_params()

In [ ]:
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=1, peak_shapes=["gaussian"], fd=False, convolve=False)
params = model.make_params(p0_center=5.0, p0_width=0.2, p0_height=3.0)
params

In [ ]:
result = model.fit(y, x=x, params=params, weights=1 / yerr)
_ = result.plot()

We can also plot components.

In [ ]:
comps = result.eval_components()
plt.errorbar(x, y, yerr, fmt="o", zorder=-1, alpha=0.3)
plt.plot(x, result.eval(), label="Best fit")
plt.plot(x, comps["1Peak_p0"], "--", label="Peak")
plt.plot(x, comps["1Peak_bkg"], "--", label="Background")
plt.legend()

Now, let us try fitting MDCs cut from simulated data with multiple Lorentzian peaks, convolved with a common instrumental resolution.

In [ ]:
data = generate_data(bandshift=-0.2, count=5e8, seed=1).T
cut = data.qsel(ky=0.3)
cut.qplot(colorbar=True)

In [ ]:
mdc = cut.qsel(eV=0.0)
mdc.qplot()

First, we define the model and set the initial parameters.

In [ ]:
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=4, peak_shapes=["lorentzian"], fd=False, convolve=True)

params = model.make_params(
    p0_center=-0.6,
    p1_center=-0.45,
    p2_center=0.45,
    p3_center=0.6,
    p0_width=0.02,
    p1_width=0.02,
    p2_width=0.02,
    p3_width=0.02,
    lin_bkg={"value": 0.0, "vary": False},
    const_bkg=0.0,
    resolution=0.03,
)
params

Then, we can fit the model to the data:

In [ ]:
result = model.fit(mdc, x=mdc.kx, params=params)
result.plot()
result

In [ ]:
result_ds = mdc.modelfit("kx", model, params=params)
result_ds

In [ ]:
from erlab.io.exampledata import generate_gold_edge
from erlab.analysis.fit.models import FermiEdgeModel

gold = generate_gold_edge(temp=100, Eres=0.02, count=5e5, seed=1)
gold.qplot(cmap="Greys")

We first select ± 0.2 eV around the Fermi level and fit the model across the energy
axis for every EDC.

In [ ]:
gold_selected = gold.sel(eV=slice(-0.2, 0.2))
result_ds = gold_selected.modelfit(
    coords="eV",
    model=FermiEdgeModel(),
    params={"temp": {"value": 100.0, "vary": False}},
    guess=True,
)
result_ds

Let's plot the fitted parameters as a function of angle!

In [ ]:
gold.qplot(cmap="Greys")
plt.errorbar(
    gold_selected.alpha,
    result_ds.modelfit_coefficients.sel(param="center"),
    result_ds.modelfit_stderr.sel(param="center"),
    fmt=".",
)

In [ ]:
from erlab.analysis.fit.models import FermiEdge2dModel

gold_norm = gold_selected / gold_selected.mean("eV")
result_ds = gold_norm.T.modelfit(
    coords=["eV", "alpha"],
    model=FermiEdge2dModel(),
    params={"temp": {"value": 100.0, "vary": False}},
    guess=True,
)
result_ds

Let's plot the fit results and the residuals.

In [ ]:
best_fit = result_ds.modelfit_best_fit.transpose(*gold_norm.dims)

fig, axs = eplt.plot_slices(
    [gold_norm, best_fit, best_fit - gold_norm],
    figsize=(4, 5),
    cmap=["viridis", "viridis", "bwr"],
    norm=[plt.Normalize(), plt.Normalize(), eplt.CenteredPowerNorm(1.0, vcenter=0)],
    colorbar="all",
    hide_colorbar_ticks=False,
    colorbar_kw={"width": 7},
)
eplt.set_titles(axs, ["Data", "FermiEdge2dModel", "Residuals"])

In [ ]:
# Define angle coordinates for 2D data
alpha = np.linspace(-5.0, 5.0, 100)
beta = np.linspace(-1.0, 1.0, 3)

# Center of the peaks along beta
center = np.array([-2.0, 0.0, 2.0])[:, np.newaxis]

# Gaussian peak on a linear background
y = -0.1 * alpha + 2 + 3 * np.exp(-((alpha - center) ** 2) / (2 * 1**2))

# Add some noise with fixed seed for reproducibility
rng = np.random.default_rng(5)
yerr = np.full_like(y, 0.1)
y = rng.normal(y, yerr)

# Transform to DataArray
darr = xr.DataArray(y, dims=["beta", "alpha"], coords={"beta": beta, "alpha": alpha})
darr.qplot()

In [ ]:
result_ds = darr.modelfit(
    coords="alpha",
    model=GaussianModel() + LinearModel(),
    params={
        "center": xr.DataArray([-2, 0, 2], coords=[darr.beta]),
        "slope": -0.1,
    },
)
result_ds

Let's overlay the fitted peak positions on the data.

In [ ]:
result_ds.modelfit_data.qplot()
result_center = result_ds.sel(param="center")
plt.plot(result_center.modelfit_coefficients, result_center.beta, '.-')

In [ ]:
result_ds = darr.modelfit(
    coords="alpha",
    model=GaussianModel() + LinearModel(),
    params={
        "center": {
            "value": xr.DataArray([-2, 0, 2], coords=[darr.beta]),
            "min": -5.0,
            "max": xr.DataArray([0, 2, 5], coords=[darr.beta]),
        },
        "slope": -0.1,
    },
)
result_ds

In [ ]:
result_ds.qshow(plot_components=True)

In [ ]:
from erlab.io.exampledata import generate_gold_edge
plt.rcParams["figure.constrained_layout.use"] = True

In [ ]:
import erlab.analysis as era
import erlab.plotting.erplot as eplt

gold = generate_gold_edge(temp=100, seed=1)
result = era.gold.poly(
    gold,
    angle_range=(-15, 15),
    eV_range=(-0.2, 0.2),
    temp=100.0,
    vary_temp=False,
    degree=2,
    plot=True,
)

In [ ]:
era.correct_with_edge(gold, result).qplot(cmap="Greys")
eplt.fermiline()

In [ ]:
from erlab.analysis.fit.minuit import Minuit
from erlab.analysis.fit.models import MultiPeakModel

model = MultiPeakModel(npeaks=4, peak_shapes=["lorentzian"], fd=False, convolve=True)

m = Minuit.from_lmfit(
    model,
    mdc,
    mdc.kx,
    p0_center=-0.6,
    p1_center=-0.45,
    p2_center=0.45,
    p3_center=0.6,
    p0_width=0.02,
    p1_width=0.02,
    p2_width=0.02,
    p3_width=0.02,
    p0_height=1500,
    p1_height=50,
    p2_height=50,
    p3_height=1500,
    lin_bkg={"value": 0.0, "vary": False},
    const_bkg=0.0,
    resolution=0.03,
)

m.migrad()
m.minos()
m.hesse()

In [ ]:
m.interactive()